# Tokenization: Building BPE from Scratch

In this notebook you'll:
- Build a character-level tokenizer and see why sequences get too long
- Build a word-level tokenizer and see the unknown-word problem
- Implement BPE's core operations: count pairs, merge pairs, train the merge table
- Encode new text using your learned merge table
- Decode token IDs back to text losslessly

**For each exercise, PREDICT the output before running the cell.**

In [ ]:
from collections import Counter

# Reproducible results
import random
random.seed(42)

# A small corpus to work with
corpus = [
    "low low low low low",
    "lower lower newest newest",
    "the cat sat on the mat",
    "the dog sat on the log",
    "the newest cat is the lowest",
]

---

## Exercise 1: Character-Level Tokenization (Guided)

The simplest tokenizer: split text into individual characters. Each character gets an ID.

**Before running, predict:** How many tokens will "The cat sat on the mat" produce? (Count carefully — spaces are characters too.) What about the vocabulary size for our corpus?

In [ ]:
def char_tokenize(text: str) -> list[str]:
    """Split text into individual characters."""
    return list(text)


def build_char_vocab(texts: list[str]) -> dict[str, int]:
    """Build a vocabulary mapping each unique character to an integer ID."""
    chars = set()
    for text in texts:
        chars.update(list(text))
    return {ch: i for i, ch in enumerate(sorted(chars))}


# Test it
test_text = "The cat sat on the mat"
char_tokens = char_tokenize(test_text)
char_vocab = build_char_vocab(corpus)

print(f"Text: '{test_text}'")
print(f"Tokens: {char_tokens}")
print(f"Number of tokens: {len(char_tokens)}")
print(f"Vocabulary size: {len(char_vocab)}")
print(f"Vocabulary: {char_vocab}")

**Result:** "The cat sat on the mat" = 22 tokens (every character including spaces). The vocabulary is tiny (~20 characters for this corpus), but sequences are long. The model would need to learn that T-h-e means "the" from scratch.

---

## Exercise 2: Word-Level Tokenization (Supported)

The other extreme: split on spaces. Each word gets an ID.

**Task:** Implement `word_tokenize` and `build_word_vocab` by filling in the TODOs.

**Then try:** What happens if you tokenize "ChatGPT is amazing" with a vocabulary built from our corpus?

In [ ]:
def word_tokenize(text: str) -> list[str]:
    """Split text on whitespace."""
    return ____  # FILL IN: one line


def build_word_vocab(texts: list[str]) -> dict[str, int]:
    """Build a vocabulary mapping each unique word to an integer ID."""
    words = set()
    for text in texts:
        words.update(____) # FILL IN: add all words from this text
    vocab = {word: i for i, word in enumerate(sorted(words))}
    vocab["[UNK]"] = len(vocab)  # Unknown token
    return vocab


# Test it
word_tokens = word_tokenize(test_text)
word_vocab = build_word_vocab(corpus)

print(f"Text: '{test_text}'")
print(f"Tokens: {word_tokens}")
print(f"Number of tokens: {len(word_tokens)}")
print(f"Vocabulary size: {len(word_vocab)}")
print(f"\nVocabulary: {word_vocab}")

# Now try a sentence with unknown words
novel_text = "ChatGPT is amazing"
novel_tokens = word_tokenize(novel_text)
encoded = [word_vocab.get(w, word_vocab["[UNK]"]) for w in novel_tokens]
print(f"\nNovel text: '{novel_text}'")
print(f"Tokens: {novel_tokens}")
print(f"Encoded: {encoded}")
print(f"Notice: 'ChatGPT' and 'amazing' both map to [UNK] = {word_vocab['[UNK]']}")
print(f"The model cannot distinguish between them.")

<details>
<summary>💡 Solution</summary>

The simplest word tokenizer just splits on whitespace. The vocabulary builder collects all unique words from the corpus.

```python
def word_tokenize(text: str) -> list[str]:
    return text.split()

def build_word_vocab(texts: list[str]) -> dict[str, int]:
    words = set()
    for text in texts:
        words.update(text.split())
    vocab = {word: i for i, word in enumerate(sorted(words))}
    vocab["[UNK]"] = len(vocab)
    return vocab
```

**Key insight:** "ChatGPT" and "amazing" both become `[UNK]`. The model sees them as identical — all information about unknown words is destroyed.

</details>

### Compare the approaches

Run this cell to see the tradeoff side by side:

In [ ]:
print("=" * 60)
print(f"{'Character-level':>20} | {'Word-level':>20}")
print("=" * 60)
print(f"{'Vocab size:':>20} {len(char_vocab):>4} | {'Vocab size:':>20} {len(word_vocab):>4}")
print(f"{'Tokens for test:':>20} {len(char_tokens):>4} | {'Tokens for test:':>20} {len(word_tokens):>4}")
print(f"{'OOV words:':>20} {'None':>4} | {'OOV words:':>20} {'Many':>4}")
print("=" * 60)
print("\nWe need something in between...")

---

## Exercise 3: BPE — Count Adjacent Pairs (Supported)

BPE starts with characters and iteratively merges the most common adjacent pair.

First, we need a function to count all adjacent pairs in a token sequence.

**Task:** Implement `get_pair_counts` that takes a list of tokens and returns a Counter of adjacent pairs.

**Example:** `["l", "o", "w", "l", "o", "w"]` should return `{("l", "o"): 2, ("o", "w"): 2, ("w", "l"): 1}`

In [ ]:
def get_pair_counts(tokens: list[str]) -> Counter:
    """Count all adjacent token pairs.
    
    Args:
        tokens: A list of string tokens
    
    Returns:
        Counter mapping (token_a, token_b) tuples to their frequency
    """
    pairs = Counter()
    for i in range(len(tokens) - 1):
        pair = ____  # FILL IN: create a tuple of adjacent tokens
        pairs[pair] += 1
    return pairs


# Test it
test_tokens = list("low low")
print(f"Tokens: {test_tokens}")
print(f"Pair counts: {get_pair_counts(test_tokens)}")
print()

# Try with the full corpus (joined)
full_text = " ".join(corpus)
full_tokens = list(full_text)
all_pairs = get_pair_counts(full_tokens)
print(f"Top 10 pairs in corpus:")
for pair, count in all_pairs.most_common(10):
    print(f"  {pair} : {count}")

<details>
<summary>💡 Solution</summary>

We iterate through the token list, looking at each adjacent pair `(tokens[i], tokens[i+1])`. A tuple of two tokens forms the key, and we count how many times each pair appears.

```python
def get_pair_counts(tokens: list[str]) -> Counter:
    pairs = Counter()
    for i in range(len(tokens) - 1):
        pair = (tokens[i], tokens[i + 1])
        pairs[pair] += 1
    return pairs
```

**Key insight:** The most frequent pair will be the first merge. BPE is greedy — it always merges the most common pair first.

</details>

---

## Exercise 4: BPE — Merge a Pair (Supported)

Once we've identified the most frequent pair, we need to merge all occurrences.

**Task:** Implement `merge_pair` that:
1. Scans through the token list
2. Wherever it finds `pair[0]` followed by `pair[1]`, replaces both with `pair[0] + pair[1]`
3. Returns the new token list

**Example:** `merge_pair(["l", "o", "w"], ("l", "o"))` should return `["lo", "w"]`

In [ ]:
def merge_pair(tokens: list[str], pair: tuple[str, str]) -> list[str]:
    """Replace all occurrences of a pair with a merged token.
    
    Args:
        tokens: Current token list
        pair: The (token_a, token_b) pair to merge
    
    Returns:
        New token list with all occurrences of the pair merged
    """
    new_tokens = []
    i = 0
    while i < len(tokens):
        # Check if current position matches the pair
        if i < len(tokens) - 1 and tokens[i] == pair[0] and tokens[i + 1] == pair[1]:
            new_tokens.append(____)  # FILL IN: the merged token
            i += ____               # FILL IN: skip both tokens
        else:
            new_tokens.append(tokens[i])
            i += 1
    return new_tokens


# Test it
test = list("low low low")
print(f"Before: {test}")

merged = merge_pair(test, ("l", "o"))
print(f"After merging ('l', 'o'): {merged}")

merged2 = merge_pair(merged, ("lo", "w"))
print(f"After merging ('lo', 'w'): {merged2}")

<details>
<summary>💡 Solution</summary>

The key is using a while loop (not for) because when we merge a pair, we skip ahead by 2 instead of 1. A for loop with `range` would visit every index, but after a merge we need to jump past the second token.

```python
def merge_pair(tokens: list[str], pair: tuple[str, str]) -> list[str]:
    new_tokens = []
    i = 0
    while i < len(tokens):
        if i < len(tokens) - 1 and tokens[i] == pair[0] and tokens[i + 1] == pair[1]:
            new_tokens.append(pair[0] + pair[1])  # Concatenate the pair
            i += 2                                  # Skip both tokens
        else:
            new_tokens.append(tokens[i])
            i += 1
    return new_tokens
```

**Note:** This is a simple linear scan. Production tokenizers use more efficient data structures, but the logic is identical.

</details>

---

## Exercise 5: BPE — The Training Loop (Supported)

Now combine `get_pair_counts` and `merge_pair` into the full BPE training loop.

**Task:** Implement `train_bpe` that:
1. Starts with character-level tokens
2. Repeats `num_merges` times:
   - Count all pairs
   - Find the most frequent pair
   - If the most frequent pair appears fewer than 2 times, stop
   - Merge that pair in the token list
   - Record the merge
3. Returns the final tokens and the list of merges

**Hints:**
- Use `Counter.most_common(1)` to get the most frequent pair
- The merge list is important — it's what we'll use for encoding new text later

In [ ]:
def train_bpe(
    text: str, num_merges: int = 20
) -> tuple[list[str], list[tuple[str, str]]]:
    """Train BPE on a text corpus.
    
    Args:
        text: The training text
        num_merges: Maximum number of merges to perform
    
    Returns:
        Tuple of (final_tokens, merge_list)
        - final_tokens: the text after all merges
        - merge_list: ordered list of (pair_a, pair_b) merges
    """
    # TODO: Start with character-level tokens
    # tokens = ...
    # merges = []
    
    # TODO: For each merge step:
    #   1. Count pairs
    #   2. Find most frequent
    #   3. If count < 2, stop
    #   4. Merge the pair
    #   5. Record the merge
    #   6. (Optional) Print progress
    
    pass  # Replace with your implementation


# Train on our corpus
full_text = " ".join(corpus)
final_tokens, merges = train_bpe(full_text, num_merges=20)

print(f"Original length (chars): {len(list(full_text))}")
print(f"Final length (tokens):   {len(final_tokens)}")
print(f"Compression: {1 - len(final_tokens)/len(list(full_text)):.1%}")
print(f"\nMerges learned ({len(merges)}):")
for i, merge in enumerate(merges):
    print(f"  {i+1}. '{merge[0]}' + '{merge[1]}' -> '{merge[0] + merge[1]}'")
print(f"\nFinal tokens: {final_tokens}")

<details>
<summary>💡 Solution</summary>

The training loop is just repeated application of the two primitives: count pairs, merge the best one. The key insight is that we stop when no pair appears more than once — further merges would only create tokens seen once, which wastes vocabulary slots.

```python
def train_bpe(
    text: str, num_merges: int = 20
) -> tuple[list[str], list[tuple[str, str]]]:
    tokens = list(text)
    merges = []

    for step in range(num_merges):
        pair_counts = get_pair_counts(tokens)
        if not pair_counts:
            break
        
        best_pair, count = pair_counts.most_common(1)[0]
        if count < 2:
            break
        
        tokens = merge_pair(tokens, best_pair)
        merges.append(best_pair)
        print(f"  Step {step+1}: merge '{best_pair[0]}' + '{best_pair[1]}' -> '{best_pair[0] + best_pair[1]}' (count: {count}, tokens: {len(tokens)})")

    return tokens, merges
```

**Key insight:** Watch the order of merges. The most frequent characters merge first (spaces, common letters), then common bigrams, then common words. Linguistic structure emerges from pure frequency statistics.

</details>

---

## Exercise 6: BPE — Encode New Text (Supported)

Training learns a merge table. Encoding applies those merges (in order!) to new text.

**Task:** Implement `encode` that:
1. Starts with character-level tokens of the input text
2. Applies each merge from the merge list, in order
3. Converts tokens to integer IDs using a vocabulary

**Important:** Merges must be applied in the same order they were learned. This ensures deterministic encoding.

**Hints:**
- You already have `merge_pair` — just apply each learned merge in sequence
- Build the vocabulary from all unique tokens seen during training

In [ ]:
def build_bpe_vocab(text: str, merges: list[tuple[str, str]]) -> dict[str, int]:
    """Build vocabulary from base characters + all merge results."""
    vocab = set(text)  # All base characters
    for pair in merges:
        vocab.add(pair[0] + pair[1])  # Add each merged token
    return {token: i for i, token in enumerate(sorted(vocab, key=lambda t: (len(t), t)))}


def encode(text: str, merges: list[tuple[str, str]], vocab: dict[str, int]) -> list[int]:
    """Encode text into token IDs using learned BPE merges.
    
    Args:
        text: The text to encode
        merges: Ordered list of learned merges
        vocab: Token-to-ID mapping
    
    Returns:
        List of integer token IDs
    """
    # TODO: Start with character-level tokens
    # tokens = ...
    
    # TODO: Apply each merge in order
    # for merge in merges:
    #     tokens = ...
    
    # TODO: Convert tokens to IDs
    # ids = ...
    
    pass  # Replace with your implementation


# Build vocabulary
vocab = build_bpe_vocab(full_text, merges)
print(f"Vocabulary ({len(vocab)} tokens):")
for token, idx in vocab.items():
    display = token.replace(' ', '\u2423')
    print(f"  {idx:3d} -> '{display}'")

# Encode some text
print("\n" + "=" * 40)
test_sentences = [
    "the cat sat on the mat",
    "the lowest",
    "newer",
    "ChatGPT",
]

for sentence in test_sentences:
    tokens_list = list(sentence)
    for merge in merges:
        tokens_list = merge_pair(tokens_list, merge)
    ids = [vocab.get(t, -1) for t in tokens_list]
    print(f"\n'{sentence}'")
    print(f"  Tokens: {tokens_list}")
    print(f"  IDs:    {ids}")

<details>
<summary>💡 Solution</summary>

Encoding is the reverse of training: start from characters, then replay the exact same merges in order. The merge order matters because earlier merges create tokens that later merges depend on.

```python
def encode(text: str, merges: list[tuple[str, str]], vocab: dict[str, int]) -> list[int]:
    tokens = list(text)
    for merge in merges:
        tokens = merge_pair(tokens, merge)
    ids = [vocab.get(t, -1) for t in tokens]
    return ids
```

**Key insight:** Encoding is deterministic. Same text + same merge table = same token IDs, every time. The merge order is critical — applying merges in a different order could produce different results.

**Notice:** "ChatGPT" gets split into characters since none of its character pairs were in our tiny training corpus. A real tokenizer trained on internet text would handle it better. But it doesn't become `[UNK]` — it degrades gracefully to character-level.

</details>

---

## Exercise 7: BPE — Decode Back to Text (Supported)

Decoding is simple: map IDs back to tokens, then concatenate.

**Task:** Implement `decode` that converts a list of integer IDs back to text.

In [ ]:
def decode(ids: list[int], vocab: dict[str, int]) -> str:
    """Decode token IDs back to text.
    
    Args:
        ids: List of integer token IDs
        vocab: Token-to-ID mapping (same one used for encoding)
    
    Returns:
        Reconstructed text string
    """
    # TODO: Build reverse vocabulary (ID -> token)
    # id_to_token = ...
    
    # TODO: Map each ID to its token and join
    # text = ...
    
    pass  # Replace with your implementation


# Test roundtrip: encode then decode
for sentence in test_sentences:
    ids = encode(sentence, merges, vocab)
    reconstructed = decode(ids, vocab)
    matches = "OK" if reconstructed == sentence else "MISMATCH"
    print(f"[{matches}] '{sentence}' -> {ids} -> '{reconstructed}'")

<details>
<summary>💡 Solution</summary>

Decoding just reverses the vocabulary mapping and concatenates. Because BPE tokens are subword pieces that fit together exactly, simple string concatenation reconstructs the original text perfectly.

```python
def decode(ids: list[int], vocab: dict[str, int]) -> str:
    id_to_token = {i: token for token, i in vocab.items()}
    tokens = [id_to_token[i] for i in ids]
    return "".join(tokens)
```

**Key insight:** Decoding is lossless. `decode(encode(text)) == text` always. This is a requirement for any tokenizer — you must be able to reconstruct the original text perfectly.

</details>

---

## Exercise 8: Put It All Together (Independent)

You now have all the pieces. Let's use them to explore how BPE behaves on different inputs.

**Tasks:**
1. Train BPE on a larger corpus (provided below) with 40 merges
2. Encode several test sentences and examine the token boundaries
3. Compare token counts for English vs. completely novel words
4. Try encoding code and see how it differs from prose

In [ ]:
# A somewhat larger corpus for more interesting merges
larger_corpus = """
the cat sat on the mat the cat sat on the mat the cat is the best cat
the dog sat on the log the dog sat on the log the dog is the best dog
the lowest price is the newest offer the lowest price is the newest offer
lower lower lower lower newest newest newest newest
the best thing is the newest thing the best thing is the newest thing
this is a test this is a test this is a test this is a test
learning is the best way to understand learning is the best way to understand
the transformer model is a newer model the transformer model is a newer model
""".strip()

# TODO: Train BPE with more merges
# final_tokens, merges = train_bpe(larger_corpus, num_merges=40)
# vocab = build_bpe_vocab(larger_corpus, merges)

# TODO: Encode these test sentences and examine the results
# test_cases = [
#     "the cat sat on the mat",         # Familiar sentence
#     "the newest transformer model",    # Mix of common and less common
#     "xyzzy abcdef",                    # Completely novel words
#     "print('hello world')",            # Code
#     "123 + 456 = 579",                 # Arithmetic
# ]

# for text in test_cases:
#     ids = encode(text, merges, vocab)
#     # Apply merges to see token boundaries
#     tokens = list(text)
#     for merge in merges:
#         tokens = merge_pair(tokens, merge)
#     print(f"'{text}'")
#     print(f"  -> {tokens}  ({len(tokens)} tokens)")
#     print()

<details>
<summary>💡 Solution</summary>

The key insight is that BPE performance depends on training data. Frequent substrings in the training corpus get merged into single tokens, so familiar text compresses well. Novel words (like "xyzzy") stay at character-level because those character pairs were never frequent enough to merge.

```python
# Train BPE with more merges
final_tokens, merges = train_bpe(larger_corpus, num_merges=40)
vocab = build_bpe_vocab(larger_corpus, merges)

# Encode test cases
test_cases = [
    "the cat sat on the mat",         # Familiar sentence
    "the newest transformer model",    # Mix of common and less common
    "xyzzy abcdef",                    # Completely novel words
    "print('hello world')",            # Code
    "123 + 456 = 579",                 # Arithmetic
]

for text in test_cases:
    ids = encode(text, merges, vocab)
    # Apply merges to see token boundaries
    tokens = list(text)
    for merge in merges:
        tokens = merge_pair(tokens, merge)
    print(f"'{text}'")
    print(f"  -> {tokens}  ({len(tokens)} tokens)")
    print()
```

**What to notice:** Familiar phrases like "the cat" compress into few tokens. Novel words like "xyzzy" stay as individual characters. Code and numbers typically tokenize poorly because they weren't frequent in training data. This is why real tokenizers train on diverse corpora.

</details>

---

## Key Takeaways

1. **Character-level** tokenization gives tiny vocabulary and no OOV, but very long sequences. The model must learn spelling from scratch.
2. **Word-level** tokenization gives compact sequences, but unknown words are invisible (`[UNK]`).
3. **BPE (subword)** is the middle ground. Common words stay whole, rare words split into recognizable pieces. It degrades gracefully — unknown words become characters, not `[UNK]`.
4. **The merge table IS the tokenizer.** Training = count pairs + merge the most frequent, repeat. Encoding = replay merges in order. Decoding = map IDs to tokens and concatenate.
5. **Encoding is deterministic.** Same text + same merge table = same output, every time. The merge order is critical.

Your five functions (`get_pair_counts`, `merge_pair`, `train_bpe`, `encode`, `decode`) implement the core algorithm behind GPT's tokenizer in ~50 lines of Python.